In [1]:
#pip install requests
#pip install DateTime
#pip install beautifulsoup4
#pip install python-telegram-bot --upgrade

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from telegram.ext import Updater
from datetime import datetime

starttime = time.time()
url = 'https://www.deraktionaer.de/'

Python 3.8.3


In [6]:
def get_article_information(url,
                            tag_parent, tag_child,
                            class_link,
                            href):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if href == False:
        article = soup.findAll(tag_parent, {'class': class_link})[:1]
        for annoucement in article:
            description = annoucement.find(tag_child)
            return description.text
    else:
        article = soup.findAll(tag_parent, {'class': class_link})[:3]
        for reference in article:
            href = reference.find(tag_child).attrs['href']
            return url + href
    
def send_push(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '<TOKEN>'
    bot_chat_id = '<CHAT_ID'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

def combine_crawled_info(description, timestamp, reference):
    bot_message = f'{description}\n\n{reference}\n\n{timestamp}'
    
    return bot_message

In [16]:
def crawl_intervall():
    list_of_timestamp = []
    
    # The bot will check every 60 seconds if a new article was posted.
    # In order to only call once the python script from google cloud,
    # the script will stop at 21:30 and restarts at 6:00. 
    
    # In order to start the script on linux machine automated, check for the "crontab"
    while True:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        # Set the time, when the bot should go to sleep
        end = '21:30:00'

        if current_time <= end:
            description = get_article_information(url, 'div', 'h2',
                                'article-list-text-container pure-u-1-1 pure-u-sm-16-24 pure-u-md-1-1 pure-u-lg-12-24', False)
            timestamp = get_article_information(url, 'div', 'small',
                                'article-list-text-container pure-u-1-1 pure-u-sm-16-24 pure-u-md-1-1 pure-u-lg-12-24', False)
            reference = get_article_information(url, 'div', 'a',
                                'article-list-image-container pure-u-1-1 pure-u-sm-8-24 pure-u-md-1-1 pure-u-lg-12-24', True)

            list_of_timestamp.append(timestamp)
            
            if len(list_of_timestamp) == 2:
                if list_of_timestamp[0] == list_of_timestamp[1]:
                    list_of_timestamp.remove(list_of_timestamp[0])
                else:
                    list_of_timestamp.clear()
                    list_of_timestamp.append(timestamp)

                    send_push(combine_crawled_info(description, timestamp, reference))
        else:
            print('Bot is going to slepp... zz')
            break

        time.sleep(60.0 - ((time.time() - starttime) % 60.0))
        
def main():
    crawl_intervall()
    
if __name__ == '__main__':
    main()

Aktueller Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Aktueller Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Verglichener Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Aktueller Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Verglichener Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Aktueller Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Verglichener Zeitwert: Heute, 15:22 ‧ Thorsten Küfner
Bot is going to slepp..zz
